In [ ]:
import os, time, threading, datetime
from google.colab import output


DELTA_URL = "https://delta.xanmarta.workers.dev/0:/"


def initiation():
    delta_folder = "/content/delta-worker"
    work_list = ['DELTA.ipynb', 'MEGA.ipynb', 'MEGAV2W.ipynb', 'MEGADRIVE.ipynb', 'DRIVEV2.ipynb', 'DRIVEV3.ipynb', 'DRIVEV4.ipynb']
    for work in work_list:
        print(work)
        work_path = os.path.join(delta_folder, work)
        %run {work_path}
    if os.path.isdir('/content/sample_data'):
        !rm -r '/content/sample_data'


def download(filePath, overwrite=True):
    # Download to '/content/file'
    full_url = os.path.join(DELTA_URL, filePath)
    fileName = os.path.basename(filePath)
    full_path = '/content/' + fileName
    trying = 0
    while True:
        if os.path.isfile(full_path):
            if overwrite:
                !rm '$full_path'
            else:
                return True
        print('Downloading', full_url)
        !curl '$full_url' -o '$full_path'
        if not os.path.isfile(full_path):
            print('File not found on host:', full_url)
            return False
        with open(full_path, 'r') as f:
            content = f.read()
        if (not '<!DOCTYPE html>' in content) and (not 'usageLimits' in content):
            return True
        trying += 1
        if trying == 10:
            print('Download failed')
            return False
        print('Download failed. Trying again')
        time.sleep(1)


def clear_output():
    while True:
        output.clear()
        time.sleep(200)


def logmsg(type, name, state, msg):
    if download(os.path.join('PROJECT', 'log.conf'), False):
        if not os.path.isfile('/usr/bin/rclone'):
            print('----- Installing rclone -----')
            !curl https://rclone.org/install.sh | sudo bash
            output.clear()
        if not os.path.isdir('/content/log'):
            !mkdir '/content/log'
        fullmsg = f'{datetime.datetime.now()} - [{type}] [{name} {state}] - {msg}'
        !rclone --config=/content/log.conf sync TO: '/content/log'
        with open('/content/log/log.txt', 'a') as f:
            f.write(fullmsg + '\n')
        !rclone --config=/content/log.conf sync '/content/log' TO:


class BaseWorker:
    def __init__(self, name, state):
        # Implement this
        self.files = []
        self.name = name
        self.state = state
        self.path = os.path.join('WORKING', name, state)
        self.success = True
        self.endmsg = 'Done'
        self.automount = False

    def prepare(self):
        if self.automount:
            if not os.path.isdir('/content/drive'):
                print('Auto mount not satisfied')
                self.endmsg = 'Auto mount not satisfied'
                return False
        for nf in self.files:
            success = download(os.path.join(self.path, nf))
            if not success:
                print(nf, 'not found')
                self.endmsg = 'File not found: ' + nf
                return False
        return True

    def working(self):
        # Implement this
        print('Nothing to work')

    def log(self, type, msg):
        logmsg(type, self.name, self.state, msg)


# Main worker
def worker(name, state):
    initiation()
    if download(os.path.join('WORKING', name, state, 'work.txt')):
        with open('/content/work.txt', 'r') as f:
            work = f.read().strip('\n')
            if work == 'DELTA':
                runner = DeltaWorker(name, state)
            elif work == 'MEGA':
                runner = MegaWorker(name, state)
            elif work == 'MEGAV2W':
                runner = Megav2Worker(name, state)
            elif work == 'DRIVEV2':
                runner = Drivev2(name, state)
            elif work == 'DRIVEV3':
                runner = Drivev3(name, state)
            elif work == 'DRIVEV4':
                runner = Drivev4(name, state)
            else:
                print('No work to do')
                logmsg('ERROR', name, state, f'No work to do. Work name: {work}')
                return
        threading.Thread(target=clear_output).start()
        if runner.prepare():
            runner.working()
            if runner.success:
                print('----- Successfully -----')
                logmsg('SUCCESS', runner.name, runner.state, runner.endmsg)
            else:
                print('----- Unsuccessfully -----')
                logmsg('ERROR', runner.name, runner.state, runner.endmsg)
        else:
            print('Preparing faied. Error:', self.endmsg)
            logmsg('ERROR', name, state, self.endmsg)
    else:
        logmsg('ERROR', name, state, 'work.txt not found')
